# Module 4: Workloads

## Exercise 1: Managing Workloads

In this exercise, you manage workloads in your Kubernetes environment.

**Objectives**

This exercise focuses on enabling you to do the following:

  - Configure ReplicaSets
  - Work with Deployments
  - Create a DaemonSet
  - Set up a StatefulSet
  - Use Jobs and CronJobs

## Exercise Equipment

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Linux Mint 20           | jumphost    | 192.168.0.5    | user                       | Netapp1!  |


---
---

##### Task 1: Configure ReplicaSets

A ReplicaSet is a stable set of pods running at any given time. 

ReplicaSets are often used to guarantee the availability of functionality. 

In this task, you use imperative commands with declarative files to create, scale, and manage ReplicaSets.




---

Review the [exercise4task1.yaml](./exercise4task1.yaml).

<details>
<summary>exercise4task1.yaml</summary>

[exercise4task1.yaml](./exercise4task1.yaml)

```yaml
apiVersion: v1
kind: ReplicaSet
metadata:
  name: frontend
  labels:
    app: guestbook
    tier: frontend
spec:
  # modify replicas according to your case
  replicas: 3
  selector:
    matchLabels:
      tier: frontend
  template:
    metadata:
      labels:
        tier: php-redis
    spec:
      containers:
      - name: php-redis
        image: redis:7.2.3
        imagePullPolicy: IfNotPresent
        resources:
          requests:
            memory: "64Mi"
            cpu: "250m"
          limits:
            memory: "128Mi"
            cpu: "500m"

```

---

Attempt to create a ReplicaSet:


In [ ]:
kubectl create -f exercise4task1.yaml


---

Correct the error in this YAML and create the ReplicaSet. 

Save the file as **exercise4task1mod.yaml**

Consult the Solutions subfolder for this exercise if you need assistance.

<details>
<summary>Solution</summary>

[exercise4Task1.yaml](./Solutions/exercise4task1.yaml)

```yaml
apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: frontend
  labels:
    app: guestbook
    tier: frontend
spec:
  # modify replicas according to your case
  replicas: 3
  selector:
    matchLabels:
      tier: frontend
  template:
    metadata:
      labels:
        tier: frontend
    spec:
      containers:
      - name: php-redis
        image: redis:7.2.3
        imagePullPolicy: IfNotPresent
        resources:
          requests:
            memory: "64Mi"
            cpu: "250m"
          limits:
            memory: "128Mi"
            cpu: "500m"

In [ ]:
kubectl create -f exercise4task1mod.yaml

---

Check the status of running pods for the ReplicaSet:


In [ ]:
kubectl get replicasets


---

Delete one of the pods in the ReplicaSet:


In [ ]:
kubectl get pods


In [ ]:
kubectl delete pod frontend-d96xq



In [ ]:
kubectl delete pod [name-of-the-pod]


In [ ]:
kubectl get pod 



---

Answer the following questions:

How many pods are running in the ReplicaSet now? 

Why?


---

The following configurations are to be done in an interactive terminal. 

You can open a terminal using ```CTRL+` ```. 

The terminal can be moved into the Editor Area to get a bigger window and move it to the side by splitting the screen to the right.



---

**OPTIONAL**: The default Linux editor is Vim. 

If you are comfortable using Vim, you can skip this step. 

To change the kubectl editor to nano, enter the following command:

In [ ]:

export KUBE_EDITOR=nano




---

Change the image of the pods in the running ReplicaSet to **nginx:1.25-alpine-slim** :


In [ ]:
kubectl get rs 


In [ ]:
kubectl edit rs [replicaset-name]


---

When you use kubectl, you can substitute rs for replicaset.


---

Ensure that all pods run the new image. To achieve this goal, delete the pods one at a time.


In [ ]:
pods=$(kubectl get pods --selector=tier=frontend -o jsonpath='{.items[*].metadata.name}');for pod in $pods; do kubectl delete pod $pod ;done

In [ ]:
pods=$(kubectl get pods --selector=tier=frontend -o jsonpath='{.items[*].metadata.name}');
first_pod=$(echo $pods | awk '{print $1}'); kubectl describe pod $first_pod| grep -i image -b5

---

Scale up the number of pods in the ReplicaSet to five:


In [ ]:
kubectl scale rs --replicas=5 frontend


Verify that the new pod is running.


In [ ]:
kubectl get rs  frontend


---

Scale down the number of pods in the ReplicaSet to two.


In [ ]:
kubectl scale rs --replicas=2 frontend


---

Verify that only two pods are running in the ReplicaSet.


In [ ]:
kubectl get rs  frontend


---

Delete the ReplicaSet:


In [ ]:
kubectl delete rs frontend


In [ ]:
kubectl get rs  frontend


---
---

##### Task 2: Work with Deployments  

A deployment manages groups of **ReplicaSets** and provides upgrades and rollback features.

In this task, you use imperative commands and declarative files to create, scale, and manage deployments.


---

In the first several steps, you work with deployments by using imperative methods.

You should try to use the kubectl commands instead of the Kubernetes IDE
extensions for these steps.



---

Create a deployment:


In [ ]:
kubectl create deployment web --image=nginx:1.19


---

Check the status of the deployment:


In [ ]:
kubectl get deployments


---

Describe the deployment:


In [ ]:
kubectl describe deploy web

---

Answer the following question:

What is the deployment's strategy and why is it used?


---

Check the status of ReplicaSets for the deployment:


In [ ]:
kubectl get replicasets


Change the image of the deployment:


In [ ]:
kubectl set image deployment/web nginx=nginx:1.21.5


You can also edit the existing deployment object’s image:

`kubectl edit deployment/web`


---

Verify the status of the rollout:


`kubectl rollout status deployment/[deployment-name]`


In [ ]:
kubectl rollout status deployment/web

---

Check the status of the deployment:


In [ ]:
kubectl get deploy


---

Answer the following question: 

How many ReplicaSets exist now?


In [ ]:
kubectl get rs


---

Answer the following question: 

How many pods for the deployment exist now?


In [ ]:
kubectl get pods


---

Answer the following question: 

What is the pod-template-hash value for the pods?

The pod-template-hash in a deployment enables and tracks rollback functionality.


---

Change the image of the deployment to an image that does not exist:


`kubectl set image deployment/[deployment-name] nginx=nginx1.21.5`


In [ ]:
kubectl set image deployment/web nginx=nginx1.21.5


---

Set up a watch on the rollout’s status in a separate terminal:


`kubectl rollout status deployment/[deployment-name] --watch`


In [ ]:
kubectl rollout status deployment/web --watch


---


Check the status of ReplicaSets for the deployment:


In [ ]:
kubectl get replicasets


---


Check the status of pods for the deployment:


In [ ]:
kubectl get pods


---

The deployment controller stops the rollout automatically because of the incorrect
image.


---

Pause the rollout:



`kubectl rollout pause deployment/[deployment-name]`


In [ ]:
kubectl rollout pause deployment/web


---

Change the image of the deployment (correcting the wrong image)

`kubectl set image deployment/[deployment-name] nginx=nginx:1.21.5`


In [ ]:
kubectl set image deployment/web nginx=nginx:1.21.5

---

Answer the following question:

Did you create new rollout with this set image change? 

Check the terminal instance by running the watch command.


---

Resume the rollout:

`kubectl rollout resume deployment/[deployment-name]`


In [ ]:
kubectl rollout resume deployment/web

---

Check the revisions of this deployment:

`kubectl rollout history deployment/[deployment-name]`

In [ ]:
kubectl rollout history deployment/web


---

Check the status of ReplicaSets for the deployment:

`kubectl get replicasets`


In [ ]:
kubectl get replicasets

---

Answer the following question: 

What is the current revision?


---

Get details regarding Revision 2 of this deployment:

`kubectl rollout history deployment/[deployment-name] --revision=2`


In [ ]:
kubectl rollout history deployment/web --revision=2

In [ ]:
kubectl rollout history deployment/web --revision=3

---

Roll back to the previous revision:

`kubectl rollout undo deployment/[deployment-name]`


In [ ]:
kubectl rollout undo deployment/web

---

You can also roll back to a specific revision by specifying the **--to-revision**
option.


---

Verify that the rollback succeeded by reviewing the deployment, ReplicaSets, and pods.


---

Check the revisions of this deployment:

`kubectl rollout history deployment/[deployment-name]`

In [ ]:
kubectl rollout history deployment/web


---

Scale up the number of pods in the deployment to five:

`kubectl scale deployment/[deployment-name] --replicas=5`


In [ ]:
kubectl scale deployment/web --replicas=5

---

Delete the deployment:

`kubectl delete deployment [deployment-name]`


In [ ]:
kubectl delete deployment web

---

Verify that the deployment and all ReplicaSets and pods are deleted.


In [ ]:
kubectl get deployment

In [ ]:
kubectl get rs

In [ ]:
kubectl get pods

---

In the next several steps, you work with deployments by using declarative methods.


---

Try to create a deployment by using the [exercise4task2.yaml](./exercise4task2.yaml) file:

<details>
<summary>exercise4task2.yaml</summary>

[exercise4task2.yaml](./exercise4task2.yaml)

```yaml
apiVersion: apps/v1
kind: deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        imagePullPolicy: IfNotPresent
        resources:
          requests:
            memory: "64Mi"
            cpu: "250m"
          limits:
            memory: "128Mi"
            cpu: "500m"
        ports:
        - containerPort: 80

In [ ]:

kubectl apply -f exercise4task2.yaml


---

Answer the following question: 

What is the error message?


---

Fix the error. 

Save the fixed file as **exercise4task2mod.yaml**

Find the answer in the Solutions subfolder for this exercise.

<details>
<summary> Solution</summary>

[exercise4task2.yaml](./Solutions/exercise4task2.yaml)

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        imagePullPolicy: IfNotPresent
        resources:
          requests:
            memory: "64Mi"
            cpu: "250m"
          limits:
            memory: "128Mi"
            cpu: "500m"
        ports:
        - containerPort: 80

---

Create a deployment by using the updated exercise4task2mod.yaml file:


In [ ]:
kubectl apply -f exercise4task2mod.yaml


---

Check the status of the deployment:


In [ ]:
kubectl get deployments


---

Change the image for the deployment in the [exercise4task2mod.yaml](./exercise4task2mod.yaml) file to nginx:1.16.1.


---

Create a deployment by using the updated **exercise4task2mod.yaml** file:


In [ ]:

kubectl apply -f exercise4task2mod.yaml


---

Verify the deployment, ReplicaSets, and pods.


In [ ]:
kubectl get deployment

In [ ]:
kubectl get replicasets

In [ ]:
kubectl get pods

---

Delete the deployment:


`kubectl delete deployment [deployment-name]`


In [ ]:
kubectl delete deployment nginx-deployment

---

Verify that the deployment and all ReplicaSets and pods are deleted.


In [ ]:
kubectl get deployment

In [ ]:
kubectl get replicasets

In [ ]:
kubectl get pods

---

##### Task 3: Create a DaemonSet

DaemonSets ensure that all nodes that match certain criteria run an instance of the supplied pod.

DaemonSets bypass the default scheduling mechanisms and restrictions, so DaemonSets are ideal for
cluster-wide services, such as log forwarding or health monitoring. 

In this task, you create a DaemonSet by using declarative methods.



---

Review the [exercise4task3.yaml](./exercise4task3.yaml) file.

<details>
<summary>exercise4task3.yaml
</summary>

[exercise4task3.yaml](./exercise4task3.yaml)

```yaml
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: ds-example
spec:
  revisionHistoryLimit: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      nodeSelector:
        nodeType: edge
      containers:
      - name: nginx
        image: nginx:1.14.2
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 80



---

Answer the following questions: 

What are the DaemonSet’s node label requirements?

 Do any nodes in the cluster match these requirements? 
 
 Use `kubectl get node --show-labels`.


In [ ]:
kubectl get node --show-labels

---

Create the DaemonSet ds-example:

`kubectl apply -f exercise4task3.yaml`


In [ ]:
kubectl apply -f exercise4task3.yaml

---

View the current DaemonSets:


In [ ]:

kubectl get daemonsets



Because no matching nodes exist, no pods should be scheduled.


---

Label the kubwor1-1 node by using nodeType=edge:


In [ ]:
kubectl label node kubwor1-1 nodeType=edge


---

View the current DaemonSets again:


In [ ]:
kubectl get daemonsets


A single instance of the DaemonSet ds-example should now be deployed.


---

View the current pods and display their labels by using --show-labels:


In [ ]:
kubectl get pods --show-labels


Notice that the deployed pod has a **controller-revision-hash** and a **pod-template-
hash** label. 

You use the controller-revision-hash and the pod-template-hash to enable and track rollback functionality in a deployment.


---

Describe the DaemonSet to get cluster-wide status information about the DaemonSet:


In [ ]:
kubectl describe ds ds-example


---

When you use kubectl, you can substitute ds for daemonset.

---

Update the DaemonSet by adding a few additional labels to the pod template:

<details>
<summary>exercise4task3.yaml</summary>

[exercise4task3.yaml](./exercise4task3.yaml)

In [ ]:
kubectl apply -f exercise4task3.yaml


---

You can also edit the existing running values:

`kubectl edit ds ds-example`

---

Watch the pods and be sure to show the labels:


In [ ]:
kubectl get pods --watch --show-labels


New versions of the DaemonSet replace older versions as the older versions phase out one at
a time. 

DaemonSets have **updateStrategy**, which is similar to a deployment’s strategy.


---

Clean up the DaemonSet:


In [ ]:
kubectl delete ds ds-example


##### Task 4: Set up a StatefulSet

The StatefulSet controller manages pods that must persist or maintain state. Pod identity, including host name, network, and storage, can be considered persistent. 

In this task, you create, update, and roll back a StatefulSet. You also delve into the process of scheduling and updating a StatefulSet.



---

Review the [exercise4task4-1.yaml](./exercise4task4-1.yaml) file.

<details>
<summary>
exercise4task4-1.yaml
</summary>

[exercise4task4-1.yaml](./exercise4task4-1.yaml)

```yaml
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: sts-example
spec:
  replicas: 3
  revisionHistoryLimit: 3
  selector:
    matchLabels:
      app: stateful
  serviceName: app
  updateStrategy:
    type: OnDelete
  template:
    metadata:
      labels:
        app: stateful
    spec:
      containers:
      - name: nginx
        image: nginx:1.25-alpine-slim
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 80
        resources:
          requests:
            memory: "64Mi"
            cpu: "250m"
          limits:
            memory: "128Mi"
            cpu: "500m"


---

Create the StatefulSet from its definition file:


In [ ]:
kubectl apply -f exercise4task4-1.yaml


---

Immediately watch the pods as they are created:


In [ ]:
kubectl get pods --show-labels --watch


Unlike deployments and DaemonSets, StatefulSet pods are created one by one, according to
the ordinal index, which means that sts-example-0 gets fully provisioned before sts-
example-1 starts. 

Also note the controller-revision-hash label. 

This label serves the same purpose as the controller-revision-hash label in a DaemonSet or the pod-
template-hash in a deployment. 

This label helps to track the revision of the pod template and enables rollback functionality.


---

Describe the StatefulSet to learn more about its state:


In [ ]:
kubectl describe statefulset sts-example


---

Update the StatefulSet's pod template in your IDE and add a few additional labels:


In [ ]:
kubectl apply -f exercise4task4-1.yaml


You can also edit the existing running values:

`kubectl edit statefulset sts-example`


---

In a teminal window, ``` `CTRL+` ``` return to watching the pods:

You can copy the command from below:

None of the pods update to the new version of the pod.


---

Delete the sts-example-2 pod:


In [ ]:
kubectl delete pod sts-example-2


---

Immediately view the pods:


In [ ]:
kubectl get pods --show-labels


The new sts-example-2 pod should be created with the new, additional labels. 

The **OnDelete** update strategy does not generate a new iteration of the pod until the previous pod is deleted. 

This arrangement enables manual gating of the update process for the StatefulSet.


---

Update the StatefulSet in your IDE and change the Update Strategy Type to RollingUpdate:


In [ ]:
kubectl apply -f exercise4task4-1.yaml


---

Again, immediately view the pods:


In [ ]:
kubectl get pods --show-labels


The pods update sequentially in descending order, or largest to smallest, based on the pod's
ordinal index. 

So if sts-example-2 is not updated, it updates first, then sts-example-1
updates, and sts-example-0 updates last.


---

Delete the StatefulSet sts-example:


In [ ]:
kubectl delete statefulset sts-example


---

Recreate the StatefulSet by using the same manifest:


In [ ]:
kubectl apply -f exercise4task4-1.yaml


---

Like many applications in which you must consider state, your planning and use of
StatefulSets requires forethought. 

You can make this task easier by consistently using standard naming, ordered updates and deletes, and templated storage.


---

Now create a headless service, or a service without a ClusterIP
(ClusterIP=None), to use with the StatefulSet sts-example. 

Then explore how this process enables consistent service discovery.


---

Review the [exercise4task4-2.yaml](./exercise4task4-2.yaml) file.

<details>
<summary>
exercise4task4-2.yaml
</summary>

[exercise4task4-2.yaml](./exercise4task4-2.yaml)

```yaml
apiVersion: v1
kind: Service
metadata:
  name: app
spec:
  clusterIP: None
  selector:
    app: stateful
  ports:
  - protocol: TCP
    port: 80
    targetPort: 80


---

Create the headless service app by using the app=stateful selector from the previous
YAML:


In [ ]:
kubectl apply -f exercise4task4-2.yaml


---

Describe the newly created service:


In [ ]:
kubectl describe svc app


Notice that the service does not have a clusterIP but does list the pod endpoints. 

Headless services are unique in this behavior. 

This system provides consistent mapping to the individual pods.


---

These mappings take the form of an A record in the following format:

`<statefulset_name>-<ordinal>.<service_name>.<namespace>.svc.cluster.local`

You can use this mapping format consistently throughout other workloads.


---

Open a shell in the sts-example-0 pod:

(For an interactive session, run the command in a terminal)


---

Query the DNS entry for the app service:
# nslookup app.default.svc.cluster.local


In [ ]:
kubectl exec sts-example-0   -- /usr/bin/nslookup app.default.svc.cluster.local

---

Each instance of the StatefulSet returns an A record. 

Query the service directly to perform simple DNS round-robin load-balancing. 

You can ignore the NXDOMAIN lookups. 

The exercise environment has no registered external DNS services.


---


Ping the DNS entry for the app service (repeat the ping command at least three times and
notice which IP address returns for each ping):
# ping app.default.svc.cluster.local


In [ ]:
kubectl exec sts-example-0   -- /bin/ping  -c3 app.default.svc.cluster.local

In [ ]:
kubectl exec sts-example-0   -- /bin/ping  -c3 app.default.svc.cluster.local

In [ ]:
kubectl exec sts-example-0   -- /bin/ping  -c3 app.default.svc.cluster.local

---

Notice that the round-robin ordering uses pod images.
Use kubectl get pods -o wide to verify the IP addresses of the images.


In [ ]:
kubectl get pods -o wide

---

Query one of the instances directly:

`# nslookup sts-example-1.app.default.svc.cluster.local`

This feature is unique to StatefulSets. This feature enables services to interact directly with a
specific instance of a pod. If the pod updates and obtains a new IP, the DNS record
immediately points to it, enabling consistent service discovery.


In [ ]:
kubectl exec sts-example-0   -- /usr/bin/nslookup  sts-example-1.app.default.svc.cluster.local

In [ ]:
kubectl exec sts-example-0   -- /usr/bin/nslookup  sts-example-1.app.default.svc.cluster.local

In [ ]:
kubectl exec sts-example-0   -- /usr/bin/nslookup  sts-example-2.app.default.svc.cluster.local

In [ ]:
kubectl exec sts-example-0   -- /usr/bin/nslookup  sts-example-2.app.default.svc.cluster.local

In [ ]:
kubectl exec sts-example-0   -- /usr/bin/nslookup  sts-example-0.app.default.svc.cluster.local

In [ ]:
kubectl exec sts-example-0   -- /usr/bin/nslookup  sts-example-0.app.default.svc.cluster.local

---

Close the shell to the sts-example-0 pod, if using the interactive shell.


---

Clean up the objects:


In [ ]:
kubectl delete svc app


In [ ]:
kubectl delete statefulset sts-example


---

You can also use the Kubernetes IDE extension to clean up the StatefulSet and its
headless service.


---
---

##### Task 5: Use Jobs and CronJobs

The job controller ensures that one or more pods run and successfully terminate. 

The job controller is essentially a task executor that can run parallel tasks. 

CronJobs extend the job controller to enable jobs to run on a schedule. 

In this task, you create a Kubernetes job and work to understand how completions and parallelism directives manage the pods.


---

Review the [exercise4task5-1.yaml](./exercise4task5-1.yaml) file.

<details>
<summary>
exercise4task5-1.yaml
</summary>

[exercise4task5-1.yaml](exercise4task5-1.yaml)

```yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: job-example
spec:
  backoffLimit: 4
  completions: 4
  parallelism: 2
  template:
    spec:
      containers:
      - name: hello
        image: nginx:1.25-alpine-slim
        imagePullPolicy: IfNotPresent
        command: ["/bin/sh", "-c"]
        args: ["echo hello from $HOSTNAME!"]
      restartPolicy: Never


---

Instantiate the job-example:


In [ ]:
kubectl apply -f exercise4task5-1.yaml


---

Watch the pods as they are created:

Open an interactive shell to watch ``` `CTRL+` `


Only two pods are provisioned at a time, adhering to the parallelism attribute. 

This action continues until the total number of completions is satisfied. 

Also, the pods are labeled with controller-uid, which acts as a unique identification for a specific job.

When the pod provisioning finishes, the pods persist in a Completed state. 

The pods are not deleted after a job completes or failed unless the time-to-life option is set. 

See **ttlSecondsAfterFinished** field to automatically cleanup finished jobs.


---

Describe the job to see a summary of the related events:


In [ ]:
kubectl describe job job-example


---

Delete the job:


In [ ]:
kubectl delete job job-example


---

View the pods again:


In [ ]:
kubectl get pods


The job’s pods are now deleted. 

The pods are cleaned up when the job itself is removed.


---

**Summary**: Jobs are fire-and-forget, one-off tasks, batch processes, or executors for
a workflow engine. 

Jobs run to completion or terminate gracefully, adhering to the completions and parallelism directives.


---

Review the [exercise4task5-2.yaml](./exercise4task5-2.yaml) file.

This file is configured to run the previous job example every minute by using the cron schedule
"*/1 * * * *". This schedule is UTC only.

<details>
<summary>
exercise4task5-2.yaml
</summary>

[exercise4task5-2.yaml](./exercise4task5-2.yaml)

```yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: cronjob-example
spec:
  schedule: "*/1 * * * *"
  successfulJobsHistoryLimit: 2
  failedJobsHistoryLimit: 1
  jobTemplate:
    spec:
      completions: 4
      parallelism: 2
      template:
        spec:
          containers:
          - name: hello
            image: debian:11.8-slim
            imagePullPolicy: IfNotPresent
            command: ["/bin/sh", "-c"]
            args: ["echo hello from $HOSTNAME!"]
          restartPolicy: Never


---

Instantiate a CronJob:


In [ ]:
kubectl create -f exercise4task5-2.yaml


---

Give the CronJob time to run, and then list the jobs:


In [ ]:
kubectl get jobs


The format should name at least one job:

` <cronjob-name>-<unix time stamp>`


---

Notice the timestamp of the oldest job.


---

Wait a few minutes and list the jobs again:


In [ ]:
kubectl get jobs


The oldest job should disappear. 

The CronJob controller purges jobs according to the **successfulJobHistoryLimit** and **failedJobHistoryLimit** attributes. 

In this case, the list retains only the last three successful jobs.


---

Describe the CronJob cronjob-example:


In [ ]:
kubectl describe CronJob cronjob-example


The events show the job creation and deletion records.


---

Edit the CronJob cronjob-example, locate the Suspend field, and set the field to **true**:

You have to do this in an interactive shell ( ``` CTRL+` ```)

This action prevents the CronJob from firing off any future events, which can help you initially
troubleshoot an issue without having to delete the CronJob directly.


---

Delete the CronJob:


In [ ]:
kubectl delete CronJob cronjob-example


Deleting the CronJob *deletes all child jobs*. 

Use Suspend to stop the job temporarily if you are attempting to troubleshoot.



End of exercise
